# 0. Imports and Setting up Anthropic API Client

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install python-dotenv

import os
import dotenv

dotenv.load_dotenv('/content/drive/MyDrive/.env')

True

In [3]:
# Load Prompts and Problem Description
prompt1_path = '/content/drive/MyDrive/Thesis/Prompts/Prompt1_MathematicalModel.txt'
prompt2_path = '/content/drive/MyDrive/Thesis/Prompts/Prompt2_PyomoCode.txt'
problem_desc_path = '/content/drive/MyDrive/Thesis/ProblemDescriptions/LP/LP1.txt'

prompt1_file = open(prompt1_path, "r")
prompt2_file = open(prompt2_path, "r")
problem_desc_file = open(problem_desc_path, "r")

prompt1 = prompt1_file.read()
print("Prompt 1:\n", prompt1)

prompt2 = prompt2_file.read()
print("Prompt 2:\n", prompt2)

problem_desc = problem_desc_file.read()
print("Problem Description:\n", problem_desc)

Prompt 1:
 Please write a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
Prompt 2:
 Please write a python pyomo code for this optimization problem.
Use sample data where needed.
Indicate where you use sample data.
Problem Description:
 A firm from Milan sells chemical products for professional cosmetics. It is planning the production of three products, GCA, GCB and GCC, for a given period of
time by mixing two different components: C1 and C2. All the end products must
contain at least one of the two components, and not necessarily both.

For the next planning period, 10,000 l of C1 and 15,000 l of C2 are available.
The production of GCA, GCB and GCC must be scheduled to at least cover the
minimum demand level of 6,000, 7,000 and 9,000 l, respectively. It is assumed
that when chemical components are mixed, there is no loss or gain in volume.

Each chemical component, C1 and C2, has a pr

In [4]:
!pip install anthropic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.1/871.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00


In [5]:
# Importing Anthropic & Setting Headers
import anthropic

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=os.environ.get("ANTHROPIC_API_KEY"),
)

# 1. Prompt 1: Create Mathematical Model

In [6]:
message = client.messages.create(
    model="claude-3-opus-20240229",
    system=prompt1,
    max_tokens=4096,
    messages=[
        {"role": "user", "content": problem_desc}
    ]
)

response_p1 = message.content[0].text

In [7]:
# Print response
print(response_p1)

Let's formulate the mathematical optimization model for this problem.

Parameters:
- C1: Available quantity of component 1 (10,000 l)
- C2: Available quantity of component 2 (15,000 l)
- D_GCA, D_GCB, D_GCC: Minimum demand for products GCA, GCB, and GCC (6,000, 7,000, and 9,000 l, respectively)
- CE_C1, CE_C2: Proportional critical element in components C1 and C2 (0.4 and 0.2, respectively)
- CE_GCA_min: Minimum fraction of the critical element in product GCA (0.3)
- CE_GCB_max: Maximum fraction of the critical element in product GCB (0.3)
- R_GCC_min: Minimum ratio of C1 to C2 in product GCC (0.3)
- P_GCA, P_GCB, P_GCC: Profit per litre for products GCA, GCB, and GCC ($120, $135, and $155, respectively)

Decision Variables:
- X_GCA_C1, X_GCA_C2: Quantity of components C1 and C2 used in product GCA
- X_GCB_C1, X_GCB_C2: Quantity of components C1 and C2 used in product GCB
- X_GCC_C1, X_GCC_C2: Quantity of components C1 and C2 used in product GCC

Objective Function:
Maximize Total Prof

# 2. Prompt 2: Write Code based on Mathematical Model

In [8]:
message2 = client.messages.create(
    model="claude-3-opus-20240229",
    system=prompt2,
    max_tokens=4069,
    messages=[
        {"role": "user", "content": response_p1}
    ]
)

response_p2 = message2.content[0].text

In [9]:
# Print response
print(response_p2)

Great! Now let's implement the optimization model using Pyomo in Python.

```python
from pyomo.environ import *

# Create a Pyomo model
model = ConcreteModel()

# Define parameters
model.C1 = 10000
model.C2 = 15000
model.D_GCA = 6000
model.D_GCB = 7000
model.D_GCC = 9000
model.CE_C1 = 0.4
model.CE_C2 = 0.2
model.CE_GCA_min = 0.3
model.CE_GCB_max = 0.3
model.R_GCC_min = 0.3
model.P_GCA = 120
model.P_GCB = 135
model.P_GCC = 155

# Define decision variables
model.X_GCA_C1 = Var(domain=NonNegativeReals)
model.X_GCA_C2 = Var(domain=NonNegativeReals)
model.X_GCB_C1 = Var(domain=NonNegativeReals)
model.X_GCB_C2 = Var(domain=NonNegativeReals)
model.X_GCC_C1 = Var(domain=NonNegativeReals)
model.X_GCC_C2 = Var(domain=NonNegativeReals)

# Define objective function
def obj_rule(model):
    return model.P_GCA * (model.X_GCA_C1 + model.X_GCA_C2) + \
           model.P_GCB * (model.X_GCB_C1 + model.X_GCB_C2) + \
           model.P_GCC * (model.X_GCC_C1 + model.X_GCC_C2)

model.OBJ = Objective(rule=ob

# 4. Input Problem Data and Test Model Code

In [10]:
# Installing pyomo nd solver
!pip install -q pyomo
!apt-get install -y -qq glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.6 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 121918 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected pac

In [11]:
from pyomo.environ import *

# Create a Pyomo model
model = ConcreteModel()

# Define parameters
model.C1 = 10000
model.C2 = 15000
model.D_GCA = 6000
model.D_GCB = 7000
model.D_GCC = 9000
model.CE_C1 = 0.4
model.CE_C2 = 0.2
model.CE_GCA_min = 0.3
model.CE_GCB_max = 0.3
model.R_GCC_min = 0.3
model.P_GCA = 120
model.P_GCB = 135
model.P_GCC = 155

# Define decision variables
model.X_GCA_C1 = Var(domain=NonNegativeReals)
model.X_GCA_C2 = Var(domain=NonNegativeReals)
model.X_GCB_C1 = Var(domain=NonNegativeReals)
model.X_GCB_C2 = Var(domain=NonNegativeReals)
model.X_GCC_C1 = Var(domain=NonNegativeReals)
model.X_GCC_C2 = Var(domain=NonNegativeReals)

# Define objective function
def obj_rule(model):
    return model.P_GCA * (model.X_GCA_C1 + model.X_GCA_C2) + \
           model.P_GCB * (model.X_GCB_C1 + model.X_GCB_C2) + \
           model.P_GCC * (model.X_GCC_C1 + model.X_GCC_C2)

model.OBJ = Objective(rule=obj_rule, sense=maximize)

# Define constraints
def component_availability_c1(model):
    return model.X_GCA_C1 + model.X_GCB_C1 + model.X_GCC_C1 <= model.C1

def component_availability_c2(model):
    return model.X_GCA_C2 + model.X_GCB_C2 + model.X_GCC_C2 <= model.C2

def minimum_demand_gca(model):
    return model.X_GCA_C1 + model.X_GCA_C2 >= model.D_GCA

def minimum_demand_gcb(model):
    return model.X_GCB_C1 + model.X_GCB_C2 >= model.D_GCB

def minimum_demand_gcc(model):
    return model.X_GCC_C1 + model.X_GCC_C2 >= model.D_GCC

def critical_element_gca(model):
    return model.CE_C1 * model.X_GCA_C1 + model.CE_C2 * model.X_GCA_C2 >= model.CE_GCA_min * (model.X_GCA_C1 + model.X_GCA_C2)

def critical_element_gcb(model):
    return model.CE_C1 * model.X_GCB_C1 + model.CE_C2 * model.X_GCB_C2 <= model.CE_GCB_max * (model.X_GCB_C1 + model.X_GCB_C2)

def ratio_c1_c2_gcc(model):
    return model.X_GCC_C1 >= model.R_GCC_min * model.X_GCC_C2

model.C1_Availability = Constraint(rule=component_availability_c1)
model.C2_Availability = Constraint(rule=component_availability_c2)
model.GCA_MinDemand = Constraint(rule=minimum_demand_gca)
model.GCB_MinDemand = Constraint(rule=minimum_demand_gcb)
model.GCC_MinDemand = Constraint(rule=minimum_demand_gcc)
model.GCA_CriticalElement = Constraint(rule=critical_element_gca)
model.GCB_CriticalElement = Constraint(rule=critical_element_gcb)
model.GCC_RatioC1C2 = Constraint(rule=ratio_c1_c2_gcc)

# Solve the optimization model
solver = SolverFactory('glpk')
results = solver.solve(model)

# Print the optimal solution
print("Optimal Solution:")
print("X_GCA_C1 =", model.X_GCA_C1.value)
print("X_GCA_C2 =", model.X_GCA_C2.value)
print("X_GCB_C1 =", model.X_GCB_C1.value)
print("X_GCB_C2 =", model.X_GCB_C2.value)
print("X_GCC_C1 =", model.X_GCC_C1.value)
print("X_GCC_C2 =", model.X_GCC_C2.value)
print("Total Profit =", model.OBJ())

Optimal Solution:
X_GCA_C1 = 3730.76923076923
X_GCA_C2 = 2269.23076923077
X_GCB_C1 = 3500.0
X_GCB_C2 = 3500.0
X_GCC_C1 = 2769.23076923077
X_GCC_C2 = 9230.76923076923
Total Profit = 3525000.0


# 5. Correct The Model Code to Test Mathematical Model (if applicable)